In [1]:
# Install the oracledb module

# Load environment variables from .env file
import oracledb
import os
import io
import pandas as pd
from dotenv import load_dotenv 
from app.sql.querys import GET_ERROS_ATENDIMENTO , GET_QUERY_MAIN , GET_QUERY_SUCESSOS_ERROS
from app.utils.redenrizar_email import TemplateEmail
from app.utils.consultar_query import Query
from app.core.send_emails import SendEmail


In [14]:
query = Query()

df_erros = query.execute_query(GET_ERROS_ATENDIMENTO)

df_erros = df_erros.rename(columns={
     "DS_ERRO_COUNT ":"Qtd de Erros",
      "DS_ERRO": "Descrição do Erro"
})


print(df_erros)


   QT_ERROS                                  Descrição do Erro
0         2  Este registro deverá ser importado de forma ma...
1         1  Registro de Paciente sem todos os dados obriga...
2         1  Existe um procedimento (0416110010 LOBECTOMIA ...
3         3  Este registro deverá ser importado de forma ma...
4         1  Este registro deverá ser importado de forma ma...
5         1  Data da Execução do procedimento 0416010121 (0...


In [3]:
#df_query_main = query.execute_query(GET_QUERY_MAIN)
df_query_sucesso_erro = query.execute_query(GET_QUERY_SUCESSOS_ERROS)

#print(df_query_main)
print(df_query_sucesso_erro)

   SUCESSO  ERROS
0        5      7


In [15]:
df_query_main = query.execute_query(GET_QUERY_MAIN)
print(df_query_main)

   TOTAL_LEGADO  TOTAL_API  DIFERENCA
0            12         12          0


In [5]:
template_email = TemplateEmail()
html_email = template_email.gerar_html_email(df_query_main, df_query_sucesso_erro, df_erros)

📁 Template está procurando em: c:\Users\Iury\Desktop\my-python-api\app\templates
✅ Prévia do e-mail salva em: c:\Users\Iury\Desktop\my-python-api\app\utils\..\output\email_preview.html


In [12]:
def enviar_email_com_relatorio():
    # 🔹 Gerar HTML do template
    template_email = TemplateEmail()
    query = Query()

    df_diferenca = query.execute_query(GET_QUERY_MAIN)
    df_sucesso_erro = query.execute_query(GET_QUERY_SUCESSOS_ERROS)
    df_erros = query.execute_query(GET_ERROS_ATENDIMENTO)

       # 🔍 Verificação de segurança
    if df_diferenca is None or df_sucesso_erro is None or df_erros is None:
        print("❌ Um ou mais DataFrames retornaram None. Verifique as queries.")
        return
    
    html_email = template_email.gerar_html_email(
        dados_erros=df_erros,
        dados_qtd=df_sucesso_erro,
        dados_diferenca=df_diferenca
    )

    # 🔹 Criar Excel em memória
    excel_buffer = io.BytesIO()
    with pd.ExcelWriter(excel_buffer, engine="openpyxl") as writer:
        df_erros.to_excel(writer, index=False, sheet_name="Erros")
        df_sucesso_erro.to_excel(writer, index=False, sheet_name="Sucesso_vs_Erros")
        df_diferenca.to_excel(writer, index=False, sheet_name="Diferencas")

    excel_buffer.seek(0)
    arquivos = {
        "relatorio_integracao.xlsx": excel_buffer
    }

    # 🔹 Enviar o e-mail com sua classe
    email_sender = SendEmail()
    email_sender.assunto = "📊 Relatório de Integração - Atendimento"
    email_sender.destinatarios = [email_sender.email_to]  # ou outro(s)
    
    resultado = email_sender.enviar_email(mensagem=html_email, arquivos=arquivos)
    print(resultado)

In [13]:
enviar_email_com_relatorio()

📁 Template está procurando em: c:\Users\Iury\Desktop\my-python-api\app\templates


2025-04-07 14:25:42,907 - INFO - Anexo adicionado: relatorio_integracao.xlsx


✅ Prévia do e-mail salva em: c:\Users\Iury\Desktop\my-python-api\app\utils\..\output\email_preview.html


2025-04-07 14:25:44,053 - INFO - E-mail enviado com sucesso para ['suporte@inovemed.com.br']!


{'message': 'Email sent successfully'}
